In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Data Prep

In [6]:
train_df = pd.read_csv("office_train.csv")
train_df

,season,episode,episode_name,andy,angela,darryl,dwight,jim,kelly,kevin,...,paul_lieberstein,mindy_kaling,paul_feig,gene_stupnitsky,jennifer_celotta,randall_einhorn,brent_forrester,jeffrey_blitz,justin_spitzer,imdb_rating
0,Season 1,1,pilot,0,1,0,29,36,0,1,...,0,0,0,0,0,0,0,0,0,7.6
1,Season 1,2,diversity day,0,4,0,17,25,2,8,...,0,0,0,0,0,0,0,0,0,8.3
2,Season 1,5,basketball,0,3,15,25,21,0,1,...,0,0,0,0,0,0,0,0,0,8.4
3,Season 1,6,hot girl,0,3,0,28,55,0,5,...,0,1,0,0,0,0,0,0,0,7.8
4,Season 2,2,sexual harassment,0,2,9,11,16,0,6,...,0,0,0,0,0,0,0,0,0,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Season 9,4,work bus,43,5,11,40,60,0,17,...,0,0,0,0,0,0,1,0,0,7.9
111,Season 9,17,farm,5,8,1,51,6,0,12,...,1,0,0,0,0,0,0,0,0,7.5
112,Season 9,18,promos,11,32,12,37,31,0,9,...,0,0,0,0,1,0,0,0,0,8.0
113,Season 9,21,livin dream,68,30,11,54,63,0,13,...,0,0,0,0,0,0,0,1,0,8.9


In [7]:
train_df.columns

Index(['season', 'episode', 'episode_name', 'andy', 'angela', 'darryl',
       'dwight', 'jim', 'kelly', 'kevin', 'michael', 'oscar', 'pam', 'phyllis',
       'ryan', 'toby', 'erin', 'jan', 'ken_kwapis', 'greg_daniels',
       'b_j_novak', 'paul_lieberstein', 'mindy_kaling', 'paul_feig',
       'gene_stupnitsky', 'jennifer_celotta', 'randall_einhorn',
       'brent_forrester', 'jeffrey_blitz', 'justin_spitzer', 'imdb_rating'],
      dtype='object')

## Problem 2: Applying Ridge and LASSO

Ridge and LASSO are great methods for parsing through data sets with lots of predictors to find:

  1. An interpretable set of important predictors - which predictors are **signal** and which ones are just **noise**
  2. The set of parameters that minimize expected prediction error (with all the caveats that we discussed in the previous lectures)
  
Where these methods really shine for purpose 1 (and purpose 2, by construction) is when the ratio of predictors to observations approaches 1.  To see this and work through an example using pre-built software, let's try to build a model that predicts IMDB ratings for episodes of the Office (the U.S. Version).  `office_train.csv` includes IMDB ratings (`imdb_rating`) for 115 episodes of the office and a number of predictors for each episode:

  1. The season of the episode (1 - 9, which should be treated as an unordered categorical variable)
  2. The number of times main characters speak in the episode (`andy` through `jan`)
  3. The director of the episode (`ken_kwapis` through `justin_spitzer`).  There can be more than 1 director per episode, so it's not a pure categorical variable.  However, the correlation is high!
  
Let's use this data to build a predictive model for IMDB ratings and check our predictive accuracy on the heldout test set (`office_test.csv`).

For this problem, you can restrict your search to the set of standard linear models (e.g. no interactions, no basis expansions, etc.).  If you would like to try to include more terms to improve the model, you are more than welcome to try!

### Part 1 (10 pts.)

Start by limiting yourself to the standard OLS model.   

Find the regression coefficients that minimize the training error under squared error loss and use this model to compute the LOOCV estimate of the expected prediction error.

Which predictors are important?  Which ones are not?  This can be difficult to tell from the OLS estimates!

In [44]:
# OLS
train_predictor = train_df.drop(['episode', 'imdb_rating', 'episode_name'], axis=1)
train_predictor['season'] = train_predictor['season'].apply(lambda _: _[-1])
# check na
train_predictor.astype(float).isna().any(axis=None)

False

In [45]:
train_predictor = train_predictor.astype(float)

In [46]:
train_predictor.insert(0, 'intercept', np.ones(len(train_predictor)))
train_predictor

,intercept,season,andy,angela,darryl,dwight,jim,kelly,kevin,michael,...,b_j_novak,paul_lieberstein,mindy_kaling,paul_feig,gene_stupnitsky,jennifer_celotta,randall_einhorn,brent_forrester,jeffrey_blitz,justin_spitzer
0,1.0,1.0,0.0,1.0,0.0,29.0,36.0,0.0,1.0,81.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,4.0,0.0,17.0,25.0,2.0,8.0,75.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,3.0,15.0,25.0,21.0,0.0,1.0,104.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,3.0,0.0,28.0,55.0,0.0,5.0,106.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,2.0,0.0,2.0,9.0,11.0,16.0,0.0,6.0,100.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,1.0,9.0,43.0,5.0,11.0,40.0,60.0,0.0,17.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
111,1.0,9.0,5.0,8.0,1.0,51.0,6.0,0.0,12.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,1.0,9.0,11.0,32.0,12.0,37.0,31.0,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
113,1.0,9.0,68.0,30.0,11.0,54.0,63.0,0.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [48]:
train_y = train_df['imdb_rating']
train_y

0      7.6
1      8.3
2      8.4
3      7.8
4      8.2
      ... 
110    7.9
111    7.5
112    8.0
113    8.9
114    9.3
Name: imdb_rating, Length: 115, dtype: float64

In [54]:
ols_reg = sm.OLS(endog=train_y, exog=train_predictor)
ols_reg_result = ols_reg.fit()
ols_reg_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            imdb_rating   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.243
Method:                 Least Squares   F-statistic:                     2.310
Date:                Wed, 09 Feb 2022   Prob (F-statistic):            0.00170
Time:                        20:42:32   Log-Likelihood:                -51.008
No. Observations:                 115   AIC:                             160.0
Df Residuals:                      86   BIC:                             239.6
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
intercept            7.4646      0.258     28.958      0.000       6.952       7.977
season              -0.0207      0.035     -0.598      0.551      -0.089       0.048
andy              2.377e-05      0.003      0.007      0.994      -0.007       0.007
angela               0.0121      0.007      1.841      0.069      -0.001       0.025
darryl               0.0007      0.007      0.097      0.923      -0.014       0.015
dwight              -0.0028      0.003     -0.888      0.377      -0.009       0.003
jim                  0.0055      0.003      1.783      0.078      -0.001       0.012
kelly               -0.0169      0.009     -1.904      0.060      -0.034       0.001
kevin               -0.0058      0.009     -0.614      0.541      -0.025       0.013
michael              0.0044      0.002      2.687      0.009       0.001       0.008
oscar                0.0048      0.008      0.605      0.547      -0.011       0.021
pam                  0.0015      0.003      0.428      0.670      -0.005       0.008
phyllis              0.0110      0.009      1.184      0.239      -0.007       0.029
ryan                -0.0042      0.007     -0.644      0.522      -0.017       0.009
toby                -0.0007      0.006     -0.105      0.917      -0.013       0.012
erin                 0.0044      0.007      0.647      0.519      -0.009       0.018
jan                  0.0038      0.004      0.978      0.331      -0.004       0.011
ken_kwapis          -0.0341      0.162     -0.211      0.834      -0.356       0.287
greg_daniels         0.4987      0.147      3.386      0.001       0.206       0.792
b_j_novak            0.4735      0.161      2.937      0.004       0.153       0.794
paul_lieberstein     0.4808      0.156      3.082      0.003       0.171       0.791
mindy_kaling         0.4021      0.167      2.402      0.018       0.069       0.735
paul_feig            0.1442      0.154      0.934      0.353      -0.163       0.451
gene_stupnitsky      0.3680      0.156      2.361      0.020       0.058       0.678
jennifer_celotta     0.2065      0.178      1.160      0.249      -0.147       0.560
randall_einhorn      0.0085      0.165      0.052      0.959      -0.319       0.336
brent_forrester      0.4644      0.182      2.546      0.013       0.102       0.827
jeffrey_blitz        0.3813      0.186      2.053      0.043       0.012       0.750
justin_spitzer       0.3065      0.180      1.699      0.093      -0.052       0.665
==============================================================================
Omnibus:                        1.905   Durbin-Watson:                   1.688
Prob(Omnibus):                  0.386   Jarque-Bera (JB):                1.433
Skew:                           0.120   Prob(JB):                        0.488
Kurtosis:                       3.491   Cond. N

In [65]:
# loocv
nobvs = len(train_df)
train_y_predict = ols_reg_result.predict(train_predictor)
hat_diag = ols_reg_result.get_influence().summary_frame()['hat_diag']

loocv_estimate = 1/nobvs * np.sum(
    ((train_y - train_y_predict)/(np.ones(nobvs) - hat_diag)) ** 2
)

print("The LOOCV estimate is {}".format(loocv_estimate))

The LOOCV estimate is 0.25489198976118527


We can see that (p<0.05) ...

### Part 2 (20 pts.)

Now, consider ridge regression.  Using a pre-built implementation of ridge regression, train the model using a large number of possible values for $\lambda$.  

For each value of $\lambda$ used, compute the L1-norm for the estimated coefficients (e.g. $\sum |\beta_j|$ ) and plot the value of the regression coefficients against this value - there should be a separate line for each regression coefficient. (Hint: There is a built-in method for doing this in the `glmnet` package.)  Which predictors seem to be most important?  You can see these as the one with "non-zero" regression coefficients when $\lambda$ is large or the L2-norm for the estimated coefficient set is small.  If it is too difficult to see over the entire $\lambda$ path, restrict the x variable limits to the lower part of the graph with the `xlim = c(low,high)` argument.  It may still be kind of difficult to tell from the graph - ridge regression is not known for its pretty pictures!

Finally, we need to select a value of $\lambda$ that minimizes the expected prediction error.  Using $10$-fold cross validation, find a reasonable value of $\lambda$ that should minimize the expected prediction error.  You can choose the actual minimum or a slightly less complex model (smaller $\lambda$ is less complex).  Defend this choice.

Create a plot that demonstrates the regression coefficients for the ridge regression with your optimal choice of $\lambda$.  Which predictors are important?  Which ones are not?  I recommend using a sideways bar plot - you can see an example construction [here](https://dk81.github.io/dkmathstats_site/rvisual-sideways-bargraph.html).


In [72]:
import glmnet_python
from glmnet import glmnet

ridge_reg = glmnet(x = train_predictor.to_numpy(), y = train_y.to_numpy(), family = "gaussian", 
                    alpha = 0
                )

OSError: dlopen(/opt/homebrew/lib/python3.9/site-packages/glmnet_python/GLMnet.so, 0x0006): tried: '/opt/homebrew/lib/python3.9/site-packages/glmnet_python/GLMnet.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e')), '/usr/local/lib/GLMnet.so' (no such file), '/usr/lib/GLMnet.so' (no such file)